# Credit Card Fraud Detection - Complete Pipeline

This notebook demonstrates the full end-to-end pipeline for credit card fraud detection:

1. **Data Loading & Exploration**
2. **Feature Engineering**
3. **Imbalance Handling with SMOTE**
4. **Model Training** (Logistic Regression, Random Forest, XGBoost)
5. **Model Evaluation & Comparison**
6. **Threshold Tuning for High Recall**
7. **SHAP Explainability**
8. **Results Visualization**

---

## 1. Setup & Imports

In [ ]:
# Data processing
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve
)

# Imbalanced learning
from imblearn.over_sampling import SMOTE

# XGBoost
import xgboost as xgb

# SHAP for explainability
import shap

# Model persistence
import joblib

print("✓ All libraries imported successfully")

## 2. Load Dataset

**Note:** Download the Kaggle Credit Card Fraud dataset and place it in `data/creditcard.csv`

Dataset: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
# Load data
df = pd.read_csv('../data/creditcard.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

## 3. Exploratory Data Analysis

In [ ]:
# Dataset info
print("Dataset Information:")
print(f"Total transactions: {len(df):,}")
print(f"Fraud cases: {df['Class'].sum():,}")
print(f"Fraud percentage: {100 * df['Class'].mean():.3f}%")
print(f"\nMissing values: {df.isnull().sum().sum()}")

# Class distribution
print("\nClass Distribution:")
print(df['Class'].value_counts())

In [ ]:
# Visualize class imbalance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
df['Class'].value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Class Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Class (0=Legitimate, 1=Fraud)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Legitimate', 'Fraud'], rotation=0)

# Pie chart
colors = ['#2ecc71', '#e74c3c']
labels = ['Legitimate', 'Fraud']
sizes = df['Class'].value_counts().values
axes[1].pie(sizes, labels=labels, colors=colors, autopct='%1.3f%%', startangle=90)
axes[1].set_title('Class Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Amount and Time analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Amount distribution by class
for class_val, color, label in zip([0, 1], ['#2ecc71', '#e74c3c'], ['Legitimate', 'Fraud']):
    subset = df[df['Class'] == class_val]['Amount']
    axes[0, 0].hist(subset, bins=50, alpha=0.7, label=label, color=color)
axes[0, 0].set_xlabel('Amount')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Amount Distribution by Class')
axes[0, 0].legend()
axes[0, 0].set_yscale('log')

# Box plot of Amount
df.boxplot(column='Amount', by='Class', ax=axes[0, 1])
axes[0, 1].set_title('Amount by Class')
axes[0, 1].set_xlabel('Class (0=Legitimate, 1=Fraud)')
axes[0, 1].set_ylabel('Amount')

# Time distribution
for class_val, color, label in zip([0, 1], ['#2ecc71', '#e74c3c'], ['Legitimate', 'Fraud']):
    subset = df[df['Class'] == class_val]['Time']
    axes[1, 0].hist(subset, bins=50, alpha=0.7, label=label, color=color)
axes[1, 0].set_xlabel('Time (seconds)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Time Distribution by Class')
axes[1, 0].legend()

# Statistics
stats = df.groupby('Class')[['Amount', 'Time']].describe()
axes[1, 1].axis('off')
axes[1, 1].text(0.1, 0.5, str(stats), fontsize=10, family='monospace')
axes[1, 1].set_title('Statistics by Class')

plt.tight_layout()
plt.show()

## 4. Feature Engineering

In [ ]:
# Create additional features
df_eng = df.copy()

# Log transform Amount (handles skewness)
df_eng['Amount_log'] = np.log1p(df_eng['Amount'])

# Extract hour of day
df_eng['Hour'] = (df_eng['Time'] // 3600) % 24

# Day number
df_eng['Day'] = df_eng['Time'] // (3600 * 24)

# Amount bins
df_eng['Amount_bin'] = pd.qcut(df_eng['Amount'], q=5, labels=False, duplicates='drop')

print("Feature engineering complete!")
print(f"New shape: {df_eng.shape}")
print(f"\nNew features: Amount_log, Hour, Day, Amount_bin")
df_eng.head()

## 5. Train-Test Split & Scaling

In [ ]:
# Separate features and target
X = df_eng.drop(columns=['Class'])
y = df_eng['Class']

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Training set: {len(X_train)} samples ({y_train.sum()} frauds)")
print(f"Test set: {len(X_test)} samples ({y_test.sum()} frauds)")

# Scale specific features
scaler = RobustScaler()
cols_to_scale = ['Amount', 'Amount_log', 'Hour', 'Day', 'Amount_bin']

scaler.fit(X_train[cols_to_scale])
X_train[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])

print("\n✓ Features scaled successfully")

## 6. Handle Class Imbalance with SMOTE

In [ ]:
# Apply SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

print("SMOTE Resampling Results:")
print(f"Original training set: {len(X_train)} samples")
print(f"  - Legitimate: {(y_train==0).sum()}")
print(f"  - Fraud: {(y_train==1).sum()}")
print(f"\nResampled training set: {len(X_res)} samples")
print(f"  - Legitimate: {(y_res==0).sum()}")
print(f"  - Fraud: {(y_res==1).sum()}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

y_train.value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Before SMOTE', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Class')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Legitimate', 'Fraud'], rotation=0)

y_res.value_counts().plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'])
axes[1].set_title('After SMOTE', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Class')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(['Legitimate', 'Fraud'], rotation=0)

plt.tight_layout()
plt.show()

## 7. Train Models

In [ ]:
print("="*60)
print("TRAINING MODELS")
print("="*60)

# 1. Logistic Regression
print("\n[1/3] Training Logistic Regression...")
lr = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42, n_jobs=-1)
lr.fit(X_res, y_res)
print("✓ Logistic Regression trained")

# 2. Random Forest
print("\n[2/3] Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=200, 
    class_weight='balanced',
    max_depth=10,
    min_samples_split=10,
    n_jobs=-1, 
    random_state=42
)
rf.fit(X_res, y_res)
print("✓ Random Forest trained")

# 3. XGBoost
print("\n[3/3] Training XGBoost...")
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
xgb_clf = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    eval_metric='auc',
    random_state=42,
    n_jobs=-1
)
xgb_clf.fit(X_res, y_res)
print("✓ XGBoost trained")

print("\n" + "="*60)
print("ALL MODELS TRAINED SUCCESSFULLY")
print("="*60)

## 8. Model Evaluation

In [ ]:
def evaluate_model(model, X, y, name='Model', threshold=0.5):
    """Comprehensive model evaluation."""
    # Predictions
    probs = model.predict_proba(X)[:, 1]
    preds = (probs >= threshold).astype(int)
    
    print(f"\n{'='*60}")
    print(f"  {name} Evaluation (threshold={threshold})")
    print(f"{'='*60}")
    
    # Classification report
    print(classification_report(y, preds, target_names=['Legitimate', 'Fraud']))
    
    # Metrics
    roc_auc = roc_auc_score(y, probs)
    pr_auc = average_precision_score(y, probs)
    print(f"ROC-AUC: {roc_auc:.4f}")
    print(f"PR-AUC: {pr_auc:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y, preds)
    print(f"\nConfusion Matrix:")
    print(cm)
    tn, fp, fn, tp = cm.ravel()
    print(f"TN={tn}, FP={fp}, FN={fn}, TP={tp}")
    
    return {
        'name': name,
        'probs': probs,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'threshold': threshold
    }

# Evaluate all models
results = []
for model, name in [(lr, 'Logistic Regression'), (rf, 'Random Forest'), (xgb_clf, 'XGBoost')]:
    result = evaluate_model(model, X_test, y_test, name, threshold=0.5)
    results.append(result)

## 9. Threshold Tuning for High Recall

In [ ]:
def find_optimal_threshold(y_true, y_probs, target_recall=0.90):
    """Find threshold for target recall."""
    precision, recall, thresholds = precision_recall_curve(y_true, y_probs)
    
    # Find where recall >= target
    indices = np.where(recall >= target_recall)[0]
    
    if len(indices) == 0:
        return thresholds[0], precision[0], recall[0]
    
    # Pick highest precision
    best_idx = indices[np.argmax(precision[indices])]
    best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.0
    
    return best_threshold, precision[best_idx], recall[best_idx]

# Find optimal thresholds
print("="*60)
print("THRESHOLD TUNING FOR 90% RECALL")
print("="*60)

for result in results:
    thresh, prec, rec = find_optimal_threshold(y_test, result['probs'], target_recall=0.90)
    print(f"\n{result['name']}:")
    print(f"  Optimal Threshold: {thresh:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall: {rec:.4f}")
    result['optimal_threshold'] = thresh
    result['optimal_precision'] = prec
    result['optimal_recall'] = rec

## 10. Visualize Performance

In [ ]:
# ROC Curves
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
for result in results:
    fpr, tpr, _ = roc_curve(y_test, result['probs'])
    axes[0].plot(fpr, tpr, label=f"{result['name']} (AUC={result['roc_auc']:.3f})", linewidth=2)

axes[0].plot([0, 1], [0, 1], 'k--', label='Random')
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curves', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right')
axes[0].grid(alpha=0.3)

# Precision-Recall Curve
for result in results:
    precision, recall, _ = precision_recall_curve(y_test, result['probs'])
    axes[1].plot(recall, precision, label=f"{result['name']} (AUC={result['pr_auc']:.3f})", linewidth=2)

axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curves', fontsize=14, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Model comparison table
comparison_df = pd.DataFrame([{
    'Model': r['name'],
    'ROC-AUC': r['roc_auc'],
    'PR-AUC': r['pr_auc'],
    'Default Threshold': r['threshold'],
    'Optimal Threshold': r['optimal_threshold'],
    'Optimal Precision': r['optimal_precision'],
    'Optimal Recall': r['optimal_recall']
} for r in results])

print("\nModel Comparison Summary:")
print(comparison_df.to_string(index=False))

# Highlight best model
best_model = comparison_df.loc[comparison_df['PR-AUC'].idxmax(), 'Model']
print(f"\n🏆 Best Model (by PR-AUC): {best_model}")

## 11. SHAP Explainability

In [ ]:
# Use XGBoost for SHAP (best model typically)
print("Generating SHAP explanations for XGBoost...")

# Sample for speed
sample_size = 1000
X_sample = X_test.sample(n=sample_size, random_state=42)

# Create explainer
explainer = shap.TreeExplainer(xgb_clf)
shap_values = explainer.shap_values(X_sample)

print(f"✓ SHAP values computed for {sample_size} samples")

In [ ]:
# SHAP Summary Plot
shap.summary_plot(shap_values, X_sample, max_display=20)

In [ ]:
# SHAP Bar Plot (feature importance)
shap.summary_plot(shap_values, X_sample, plot_type='bar', max_display=20)

In [ ]:
# Get top features
feature_importance = pd.DataFrame({
    'feature': X_sample.columns,
    'importance': np.abs(shap_values).mean(axis=0)
}).sort_values('importance', ascending=False)

print("Top 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

In [ ]:
# SHAP Dependence Plot for top feature
top_feature = feature_importance.iloc[0]['feature']
print(f"Dependence plot for top feature: {top_feature}")

shap.dependence_plot(
    top_feature,
    shap_values,
    X_sample,
    interaction_index='auto'
)

## 12. Save Models

In [ ]:
# Save all models and artifacts
import os
os.makedirs('../models', exist_ok=True)

joblib.dump(lr, '../models/lr.joblib')
joblib.dump(rf, '../models/rf.joblib')
joblib.dump(xgb_clf, '../models/xgb.joblib')
joblib.dump(scaler, '../models/scaler.joblib')

# Save comparison results
comparison_df.to_csv('../models/model_comparison.csv', index=False)

print("✓ All models saved to ../models/")
print("  - lr.joblib")
print("  - rf.joblib")
print("  - xgb.joblib")
print("  - scaler.joblib")
print("  - model_comparison.csv")

## 13. Summary & Recommendations

### Key Findings:
- Successfully trained 3 models with SMOTE for class imbalance
- XGBoost typically achieves the best PR-AUC score
- Optimal threshold tuning achieves 90%+ recall with acceptable precision
- SHAP analysis reveals most important features for fraud detection

### Next Steps:
1. **Deploy**: Use FastAPI service (`src/api.py`) for real-time predictions
2. **Monitor**: Track model performance over time, detect data drift
3. **Retrain**: Update model monthly or when performance degrades
4. **Features**: Add merchant, location, device fingerprint data for better accuracy
5. **Ensemble**: Combine multiple models for improved robustness

### Production Recommendations:
- Set threshold based on business cost of false positives vs false negatives
- Implement A/B testing for threshold optimization
- Add human-in-the-loop for high-value transactions
- Monitor fraud patterns and update features accordingly

---
**Notebook Complete** ✓

Run the FastAPI service:
```bash
cd ..
python -m uvicorn src.api:app --reload
```

Then test predictions at: http://localhost:8000/docs